## CBCモード

In [4]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import os

def xor_bytes(a, b):
    return bytes(i ^ j for i, j in zip(a, b))

def encrypt_cbc(plaintext, key, iv):
    cipher = AES.new(key, AES.MODE_ECB)
    block_size = AES.block_size
    padded_plaintext = pad(plaintext, block_size)

    ciphertext = b''
    previous_block = iv

    for i in range(0, len(padded_plaintext), block_size):
        block = padded_plaintext[i:i+block_size]
        block = xor_bytes(block, previous_block)
        encrypted_block = cipher.encrypt(block)
        ciphertext += encrypted_block
        previous_block = encrypted_block

    return ciphertext

def decrypt_cbc(ciphertext, key, iv):
    cipher = AES.new(key, AES.MODE_ECB)
    block_size = AES.block_size
    
    plaintext = b''
    previous_block = iv

    for i in range(0, len(ciphertext), block_size):
        block = ciphertext[i:i+block_size]
        decrypted_block = cipher.decrypt(block)
        decrypted_block = xor_bytes(decrypted_block, previous_block)
        plaintext += decrypted_block
        previous_block = block

    return unpad(plaintext, block_size)

key = os.urandom(16)
iv = os.urandom(16)
plaintext = b'Hello, this is a test message for CBC mode implementation.'

ciphertext = encrypt_cbc(plaintext, key, iv)
print(f'Ciphertext: {ciphertext}')

decrypted_plaintext = decrypt_cbc(ciphertext, key, iv)
print(f'Decrypted Plaintext: {decrypted_plaintext}')

Ciphertext: b'\xfb\xc0l\xfb\x0e\x06\xca`0\x14hU@?\xc3\xd0\xf5\xbb\x06^\x01\x88\xc7*T\xa4\x11\xcd\xe4\x9cf0K}\x0bNf\xd9T\xa8\xdc\x8b\xfeV\xb33h\x12\x13\x11\xadp\xa2)\xd5\xe7\xd6\xa5\x1d\x8f\xaaq\xa2v'
Decrypted Plaintext: b'Hello, this is a test message for CBC mode implementation.'


## OFBモード

In [11]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import os

def xor_bytes(a, b):
    return bytes(i ^ j for i, j in zip(a, b))

def encrypt_ofb(plaintext, key, iv):
    cipher = AES.new(key, AES.MODE_ECB)
    block_size = AES.block_size
    
    ciphertext = b''
    feedback = iv
    
    for i in range(0, len(plaintext), block_size):
        encrypted_feedback = cipher.encrypt(feedback)
        block = plaintext[i:i + block_size]
        block = pad(block, block_size)
        encrypted_block = xor_bytes(block, encrypted_feedback)
        ciphertext += encrypted_block
        feedback = encrypted_feedback
    
    return ciphertext

def decrypt_ofb(ciphertext, key, iv):
    cipher = AES.new(key, AES.MODE_ECB)
    block_size = AES.block_size
    
    plaintext = b''
    feedback = iv
    
    for i in range(0, len(ciphertext), block_size):
        encrypted_feedback = cipher.encrypt(feedback)
        block = ciphertext[i:i + block_size]
        decrypted_block = xor_bytes(block, encrypted_feedback)
        plaintext += decrypted_block
        feedback = encrypted_feedback
    
    return unpad(plaintext, block_size)

# Example usage
key = os.urandom(16)  # AES-128
iv = os.urandom(16)   # Initialization Vector
plaintext = b'Hello, this is a test message for OFB mode implementation.'

ciphertext = encrypt_ofb(plaintext, key, iv)
print(f'Ciphertext: {ciphertext}')

decrypted_plaintext = decrypt_ofb(ciphertext, key, iv)
print(f'Decrypted Plaintext: {decrypted_plaintext}')


Ciphertext: b"\x07\x15%\xa1:M\xb3\xf7\x87|9j\xb4\x8a\xd9KK\x9a\xd5\x96\x86\xd9\xf3<\x0c\xd6\xe9\xb9\x12\xca\xbba\xadH\xa5\xdc\x7f\x0e\x12\xa2w\x8f\x1a\x97\xc7@\nB\xb2\x9a\xd8\x0f\xad\xcbv8\xfc\xc1'j.W\xda\xa2"
Decrypted Plaintext: b'Hello, this is a test message for OFB mode implementation.'


## CTRモード

In [13]:
from Crypto.Cipher import AES
import os

def xor_bytes(a, b):
    return bytes(i ^ j for i, j in zip(a, b))

def increment_counter(counter):
    counter_int = int.from_bytes(counter, byteorder='big') + 1
    return counter_int.to_bytes(len(counter), byteorder='big')

def encrypt_ctr(plaintext, key, nonce):
    cipher = AES.new(key, AES.MODE_ECB)
    block_size = AES.block_size
    ciphertext = b''
    
    counter = nonce
    
    for i in range(0, len(plaintext), block_size):
        encrypted_counter = cipher.encrypt(counter)
        block = plaintext[i:i + block_size]
        block = xor_bytes(block, encrypted_counter)
        ciphertext += block
        counter = increment_counter(counter)
    
    return ciphertext

def decrypt_ctr(ciphertext, key, nonce):
    return encrypt_ctr(ciphertext, key, nonce)  # CTR mode decryption is the same as encryption

# Example usage
key = os.urandom(16)  # AES-128
nonce = os.urandom(16)  # Nonce for CTR mode
plaintext = b'Hello, this is a test message for CTR mode implementation.'

ciphertext = encrypt_ctr(plaintext, key, nonce)
print(f'Ciphertext: {ciphertext}')

decrypted_plaintext = decrypt_ctr(ciphertext, key, nonce)
print(f'Decrypted Plaintext: {decrypted_plaintext}')


Ciphertext: b"'4\xa3xO\xf0\xb3\x9a\xd3\x8e\xd8\x07\x01\xf1\x81\x94\xd6\xf0\xe6J\xab\xfa:\xf9j\x8d\xe5\xf7\xc3\x9c1_0\x0c>.-\x0e\x1d\xaa\x94\xf0\xc2#9\xcb\xa4\xcb\xac\xb4\xe2i\x19\x08\xae\xf2\xf5\x8d"
Decrypted Plaintext: b'Hello, this is a test message for CTR mode implementation.'


## DH鍵交換

In [15]:
import random

# 素数pと生成元gを選択
p = 23
g = 5

# AliceとBobが秘密鍵を生成
a = random.randint(1, p-1)
b = random.randint(1, p-1)

# 公開鍵を生成
A = pow(g, a, p)
B = pow(g, b, p)

# 共有秘密鍵を生成
shared_secret_A = pow(B, a, p)
shared_secret_B = pow(A, b, p)

assert shared_secret_A == shared_secret_B

print(f"共通の秘密鍵: {shared_secret_A}")




共通の秘密鍵: 18
